<figure>
  <IMG SRC="Logo.png"  WIDTH=150 ALIGN="right">
</figure>

## Projections of mean sea level and tides<br>
### Six Dutch stations
<b>Prepared by: H.G. Voortman</b>


<figure>
    <IMG SRC="https://www.python.org/static/community_logos/python-powered-w-200x80.png"  WIDTH=100 ALIGN="right">
</figure>

#### Description
- Showing examples of effects in the North Sea

#### References
- Provided in report

#### Packages

In [1]:
# General packages
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3 as sq
import numpy as np
#import scipy.signal as signal
#import datetime as dt
import hvec_stat.gof as gof
import hvec_stat.general_fit as gf
#import hvec_stat.gof as gof
import hvec_stat.support as sup
import models as mdl

In [2]:
# Settings
plt.rcParams['axes.grid'] = True
figsize = (20, 18)

#### Connect databases and import data

##### Processed data Rijkswaterstaat

In [3]:
# Connect database
conn_str = os.getenv('DATAPATH') + 'RWS_processed.db'
cnxn = sq.connect(conn_str, detect_types = True)

In [4]:
pd.read_sql('SELECT * FROM sqlite_master', cnxn)

,type,name,tbl_name,rootpage,sql
0,table,const_yr,const_yr,2,"CREATE TABLE ""const_yr"" (\n""naam"" TEXT,\n ""le..."
1,index,ix_const_yr_naam_level_1,const_yr,3,"CREATE INDEX ""ix_const_yr_naam_level_1""ON ""con..."


In [5]:
# Read table with observed water levels; complete years only
sql = (
    "SELECT * "
    "FROM 'const_yr' "
    "WHERE naam IN ('Delfzijl', 'Harlingen', "
    "'Den Helder', 'IJmuiden',  "
    "'Hoek van Holland', 'Vlissingen') "
    "AND (count>650) "
    #"AND set IN ('Ftested3') "
)
df = pd.read_sql(sql, cnxn) #.groupby('YEAR').mean()

In [6]:
cnxn.close()

In [7]:
df.columns = df.columns.str.replace('_ampl', '')
df.columns

Index(['naam', 'level_1', 'z0', 'zmean', 'count', 'M2', 'S2', 'Rsq_adj',
       'MHWS', 'MLWS', 'MHWN', 'MLWN', 'year', 'year_start', 'set', 'M4', 'O1',
       'K2', 'K1', 'P1', 'N2', 'M6', 'MU2', 'L2', '2MS6', 'MS4', 'NU2', 'SA'],
      dtype='object')

In [8]:
tsplit = 2005
tstart = 1945
#test = df[df['year'].between(tsplit + 1, 2021)]
#df = df[df['year'].between(1945, tsplit)]

In [9]:
names = [
    'Delfzijl',
    'Harlingen',
    'Den Helder',
    'IJmuiden',
    'Hoek van Holland',
    'Vlissingen'
]

In [10]:
df = df[df['set'] == 'Ftested3']
#test = test[test['set'] == 'Ftested3']

#### Empirical models

In [11]:
model_list = [
    mdl.model1,
    mdl.model2,
    mdl.model3,
    #mdl.model3a,
    #mdl.model3b,
    mdl.model4,
    #mdl.model5
]

In [12]:
def get_Rsq_and_Ftest(
    f, df, method = 'Bence'):

    # Separate training and testing data
    test = df[df['year'].between(tsplit + 1, 2021)]
    df = df[df['year'].between(tstart, tsplit)]
    
    p, _, E_x, _, _, ref = gf.fit_with_uncert(
        f, df['year'], df['z0'], method)

    # Create model data for testing period
    ymodel = gf.prediction(md, test['year'], p, E_x)

    #plt.plot(df.year, df['z0'], 'rx')
    #plt.plot(test.year, test.z0, 'ko')
    #gf.plot_general_fit(md, df['year'], df['year'], df['z0'], method = 'Bence')

    xgr = np.linspace(1945, 2022, 1000)
    #plt.plot(xgr, gf.prediction(md, xgr, p, E_x))

    Rsq = gof.Rsq_adj(test['z0'], ymodel, k = len(p), method = method)
    p_F = gof.Ftest_classic(test['z0'], ymodel, k = len(p), method = method)['p']

    return p_F, Rsq    
    # Test againts test data

In [13]:
res = pd.DataFrame()

for md in model_list:
    #tmp = pd.DataFrame()
    tmp = df.groupby('naam').apply(
        lambda x: pd.Series(get_Rsq_and_Ftest(md, x, method = 'Bence')))
    
    tmp.columns = [['p', 'Rsq_adj']]
    tmp['model'] = str(md.__name__)
    res = pd.concat([res, tmp], axis = 0)

res.sort_values(by = 'naam')

c:\users\hessel voortman\onedrive - hessel voortman ec bv\packages\hvec_stat\hvec_stat\bence_method.py:54: UserWarning: Negative autocorrelation found. Correlation set to 0
  warnings.warn('Negative autocorrelation found. Correlation set to 0')
c:\users\hessel voortman\onedrive - hessel voortman ec bv\packages\hvec_stat\hvec_stat\bence_method.py:54: UserWarning: Negative autocorrelation found. Correlation set to 0
  warnings.warn('Negative autocorrelation found. Correlation set to 0')
c:\users\hessel voortman\onedrive - hessel voortman ec bv\packages\hvec_stat\hvec_stat\bence_method.py:54: UserWarning: Negative autocorrelation found. Correlation set to 0
  warnings.warn('Negative autocorrelation found. Correlation set to 0')
c:\users\hessel voortman\onedrive - hessel voortman ec bv\packages\hvec_stat\hvec_stat\bence_method.py:54: UserWarning: Negative autocorrelation found. Correlation set to 0
  warnings.warn('Negative autocorrelation found. Correlation set to 0')


,p,Rsq_adj,model
naam,,,
Delfzijl,0.677717,-0.245498,model1
Delfzijl,0.732303,-0.229350,model3
Delfzijl,0.485658,0.006862,model4
Delfzijl,0.294474,0.176703,model2
Den Helder,0.175401,0.276857,model3
Den Helder,1.000000,-0.476722,model2
Den Helder,0.279975,0.199614,model4
Den Helder,0.352714,0.044655,model1
Harlingen,1.000000,-1.494381,model2


#### End script
Prepared by HVEC lab, 2022